In [4]:
from collections import Counter
import json

tree_logs_fn = "experiments/mtco0716_B_1/tree_logs.jsonl"

tree_logs = []
with open(tree_logs_fn, "r") as f:
    for line in f:
        tree_logs.append(json.loads(line))

print(len(tree_logs))

15


In [8]:
from IPython.display import display
import numpy as np, pandas as pd

avg_strategy_advantage = {}
per_depth_strategy_advantage = {}
per_task_strategy_advantage = {}

for tree in tree_logs:
    trace = tree["trace"]
    task = tree["task"]
    id2msg = {msg["id"]: msg for msg in trace}

    # for msg in trace:
    #     if msg["role"] == "user":
    #         children = [id2msg[child_id] for child_id in msg["children"]]
    #         scores = [child["backpropagated_score"] for child in children]
    #         mean_score = np.mean(scores)
    #         for child in children:
    #             child["advantage"] = child["backpropagated_score"] - mean_score

    for msg in trace:
        if msg["role"] == "assistant":
            # print(msg["id"].ljust(20), msg["strategy_name"].ljust(20), str(msg["backpropagated_score"]).ljust(20), msg["advantage"])
            depth = msg["id"].count("A")
            # print(msg["id"], msg.keys())
            advantage = msg.get("advantage", 0)
            if msg["strategy_name"] not in avg_strategy_advantage:
                avg_strategy_advantage[msg["strategy_name"]] = []
            avg_strategy_advantage[msg["strategy_name"]].append(advantage)
            if depth not in per_depth_strategy_advantage:
                per_depth_strategy_advantage[depth] = {}
            if msg["strategy_name"] not in per_depth_strategy_advantage[depth]:
                per_depth_strategy_advantage[depth][msg["strategy_name"]] = []
            per_depth_strategy_advantage[depth][msg["strategy_name"]].append(advantage)
            if task not in per_task_strategy_advantage:
                per_task_strategy_advantage[task] = {}
            if msg["strategy_name"] not in per_task_strategy_advantage[task]:
                per_task_strategy_advantage[task][msg["strategy_name"]] = []
            per_task_strategy_advantage[task][msg["strategy_name"]].append(advantage)

results = []
for depth, strategy_advantages in per_depth_strategy_advantage.items():
    row = {"depth": depth}
    for strategy_name, advantages in strategy_advantages.items():
        row[strategy_name] = np.mean(advantages)
    results.append(row)

avg_row = {"depth": "avg"}
for strategy_name in avg_strategy_advantage:
    avg_row[strategy_name] = np.mean(avg_strategy_advantage[strategy_name])
results.append(avg_row)

df = pd.DataFrame(results).set_index("depth")
display(df)

# now do the same for per_task_strategy_advantage
results = []
for task, strategy_advantages in per_task_strategy_advantage.items():
    row = {"task": task}
    for strategy_name, advantages in strategy_advantages.items():
        row[strategy_name] = np.mean(advantages)
    results.append(row)

df = pd.DataFrame(results).set_index("task")
display(df)

,null,list_of_requirements,scope_narrowing,clarification,from_scratch,assume_mistake,consolidate,invalidate
depth,,,,,,,,
1,0.092234,0.249074,0.000000,0.067901,NaN,NaN,NaN,NaN
2,0.062747,0.005612,0.133805,0.170332,NaN,NaN,NaN,NaN
3,0.063611,NaN,NaN,NaN,0.074237,0.132791,0.112698,0.092593
4,0.051387,NaN,NaN,NaN,0.138889,0.073830,0.104716,0.085290
5,0.055074,NaN,NaN,NaN,0.057644,0.086705,0.065401,0.064815
avg,0.055219,0.081694,0.113735,0.154159,0.096518,0.086521,0.086847,0.076950


,null,clarification,from_scratch,consolidate,assume_mistake,invalidate,scope_narrowing,list_of_requirements
task,,,,,,,,
actions,0.078326,0.310185,0.066872,0.030864,0.019608,0.027778,0.013889,0.097222
code,0.049339,0.081129,0.064589,0.099642,0.091503,0.068493,0.138272,0.111111
math,0.053866,0.046296,0.092020,0.076190,0.068376,0.069717,0.166667,0.083951
database,0.058642,0.259259,0.179012,0.081129,0.125448,0.118280,0.079365,0.030864
